# Arduino IOT

This notebook describes a simple Arduino project that streams data to an Azure IOT Hub via WiFi.

Arduino is an umbrella for a set of open hardware embedded devices extensively used in the maker community. If you are unfamiliar with them but are interested in Internet Of Things (IOT) it may be worthwhile to watch a YouTube video or two, read the Wikipedia article and so forth. To cut to the chase I provide some summary bullets from the perspective of 'how do I get stuff to work?'

* The most commonly used Arduino form factor is the Arduino Uno. This device runs about $15 and it includes some nicely arranged headers that facilitate _rapid prototyping_. Headers are just rows of pin receptacles: Little wires jump from the header to an adjacent breadboard or device and the project is off and running. The most important point is that the Arduino stack makes it fairly easy to connect to devices both with physical wires and with code.
* 

From Amazon text: http://www.amazon.com/dp/B00O2LLT30?psc=1

Directions based on Arduino IDE and UNO R3 compatible board. 

In the Arduino app 

- Go to menu option "Sketch > Include Library > Manage Libraries" and install "Adafruit GFX Library" and "Adafruit SSD1306". This will get the appropriate libraries installed. 

- Go to menu option "File > Examples > Adafruit SSD1306 > ssd1306_128x64_i2c" to bring up example code 

- In the example, find the line with "display.begin(SSD1306_SWITCHCAPVCC, 0x3D);" and change it to "display.begin(SSD1306_SWITCHCAPVCC, 0x3C);" 

Once the software is ready, connect the hardware as follows: 

OLED <> UNO R3 

-------------- 

VCC <> 5V 

GND <> GND

SDA <> A4

SCL <> A5

After that, just run the sample.


This little display is great - it is half the price of the Adafruit version and does not have a multitude of wires. The display works with the Adafruit_SSD1306 Arduino library. For those unfamiliar with that library, be sure to open the .h file, look for line 54-56 which will have a few lines of #define's. Simply leave the one that has 128x64 uncommented and comment out the other display sizes. There is a jumper on the back of this display to let you select the I2C address. Mine said it was 0x78 but was actually 0x3C (the Adafruit library is set to 0x3D by default). Also, the listing says this board is 3 to 5 volts, but I know the Adafruit ones are either 3 or 5 volt. Since this board probably isn't as well built, I decided to run it only on 3.3 volts straight off my Arduino. And I also should mention the upper ~20-25% of this display is yellow and the lower section is a teal blue - it isn't a dual color display where every pixel's color is selectable. 


Works well with a little reading.

I found that you DID NOT have to adjust the header file and only need to initialize the display with display.begin(SSD1306_SWITCHCAPVCC, 0x3C);

Worked on both an Uno and a Mega.

Uno Pinout
SDA - A4
SDL - A5
GND - GND
VCC - 5v

Mega Pinout
SDA - 20
SDL - 21
GND - GND
VCC - 5v

The example code for the SSD1306 really starts to push the memory of the Uno, but the Mega has no issue loading the program.

Because it is an OLED, it will look dead until it has been initialized, which can be a bit confusing when you are first testing the screen. I thought I had a dud until I got it correctly initialized. Had no issues with 3.3v or 5v.

Tested with a 30k Thermistor.

#include <SPI.h>
#include <Wire.h>
#include <Adafruit_GFX.h>
#include <Adafruit_SSD1306.h>

#define OLED_RESET 4
Adafruit_SSD1306 display(OLED_RESET);

#if (SSD1306_LCDHEIGHT != 64)
#error("Height incorrect, please fix Adafruit_SSD1306.h!");
#endif

#define THERMISTORPIN A0 // which analog pin to connect
#define THERMISTORNOMINAL 30000 // resistance at 25 degrees C
#define TEMPERATURENOMINAL 25 // temp. for nominal resistance (almost always 25 C)
#define NUMSAMPLES 50 // how many samples to take and average, more takes longer
#define BCOEFFICIENT 4400 // The beta coefficient of the thermistor (usually 3000-4000)
#define SERIESRESISTOR 30000 // the value of the 'other' resistor

int samples[NUMSAMPLES];

void setup() {
 Serial.begin(9600);

 // connect AREF to 3.3V and use that as VCC, less noisy!
 // connect AREF to 3.3V and use that as VCC, less noisy!
 analogReference(EXTERNAL);

 // by default, we'll generate the high voltage from the 3.3v line internally! (neat!)
 display.begin(SSD1306_SWITCHCAPVCC, 0x3C); // initialize with the I2C addr 0x3D (for the 128x64)
 // init done

 // Show image buffer on the display hardware.
 // Since the buffer is intialized with an Adafruit splashscreen
 // internally, this will display the splashscreen.
 display.display();
 delay(250);

 // Clear the buffer.
 display.clearDisplay();
}

void loop() {

 uint8_t i;
 float average;
 float maxTemp;
 maxTemp = 0;
 // take N samples in a row, with a slight delay
 for (i=0; i< NUMSAMPLES; i++) {
 samples[i] = analogRead(THERMISTORPIN);
 delay(10);
 }

 // average all the samples out
 average = 0;
 for (i=0; i< NUMSAMPLES; i++) {
 average += samples[i];
 }
 average /= NUMSAMPLES;

 display.clearDisplay();
 display.setTextSize(1);
 display.setTextColor(WHITE);
 display.setCursor(0,0);

 Serial.print("Analog ");
 Serial.println(average);
 display.print("Analog ");
 display.println(average);

 // convert the value to resistance
 average = 1023 / average - 1;
 average = SERIESRESISTOR / average;
 Serial.print("R ");
 Serial.println(average);
 display.print("R-Val ");
 display.println(average);
 display.setTextSize(2);

 float steinhart;
 float fconvert;
 float convertValue;
 steinhart = average / THERMISTORNOMINAL; // (R/Ro)2
 steinhart = log(steinhart); // ln(R/Ro)
 steinhart /= BCOEFFICIENT; // 1/B * ln(R/Ro)
 steinhart += 1.0 / (TEMPERATURENOMINAL + 273.15); // + (1/To)
 steinhart = 1.0 / steinhart; // Invert
 steinhart -= 273.15; // convert to C
 Serial.print(steinhart);
 Serial.println(" *C");
 fconvert = steinhart * 1.8;
 fconvert = fconvert + 32;
 Serial.print("Temperature ");
 display.println("A0 Temp");
 Serial.print(fconvert);
 Serial.println(" *F");
 display.print(steinhart);
 display.println(" *C");
 display.print(fconvert);
 display.println(" *F");
 display.display();

 delay(250);
}
 
 
 It works just fine with the "Sainsmart Nano V3.0" with SDA connected to pin A4 of the Nano and SCL connected to pin A5.

Like the others, I used the Adafruit library and had to change the address to 0x3C to communicate with the display. If the address is incorrect it may not appear to function, so be sure to check if you are having troubles. To ensure I was using the correct address, I scanned the I2C bus using a sketch on github: "https://github.com/todbot/arduino-i2c-scanner/blob/master/I2CScanner/I2CScanner.ino". It showed "addr: 60 found!" Convert 60 to hex and you get 0x3C. Good luck! 




Excellent display for the price. Highly Recommended. Here are a few tips and tricks: Top 16 rows (0-15) are Yellow, then Next 48 are blue. Glass can be a bit fragile, so if you mount the display on something (like plexiglas), don't go too tight. Adafruit's SSD1306 library works great. If you want to display bitmap images, then search google for "lcd-image-converter" windows app. It will allow you to import a monochrome .BMP image and output a static char array that can be used in your c/c++/arduino code and passed to the ssd1306 library display.drawBitmap() function. To create a monochrome image from a color one, use photoshop functions "desaturate, threshold, then reduce its size (to less <= 128x64pixels) using "image size' (resample using nearest neighbor (preserve hard edges))) . Once you have a monochrome image, using lcd-image-converter, import your image, then under options, use the "monochrome" preset, type=monochrome/diffuse dither, main scan direction=top to bottom, line scan=fwd, image block size=8 bit, then click "show preview" and copy this code into your static array following the adafruit ssd1306_128x64_i2c.ino/ testdrawbitmap() function example. Lastly, OLEDs are susceptible to "screen burn-in" -- read about it in wikipedia. Use a screensaver or blank your screen on occasion -- that is, if you care. 


